### **TF-IDF: Exercises**
 
- Humans 👦 show different emotions/feelings based on the situations and communicate them through facial expressions or in form of words.
 
- In Social Media like Twitter and Instagram, many people express their views through comments about a particular event/scenario and these comments may address the feelings like sadness, happiness, joy, sarcasm, fear, and many other.
 
- For a given comment/text, we are going to use classical NLP techniques and classify under which emotion that particular comment belongs!
 
- We are going to use techniques like Bag of grams, n-grams, TF-IDF, etc. for text representation and apply different classification algorithms.

### **About Data: Emotion Detection**

Credits: https://www.kaggle.com/datasets/praveengovi/emotions-dataset-for-nlp


- This data consists of two columns.
        - Comment
        - Emotion
- Comment are the statements or messages regarding to a particular event/situation.

- Emotion feature tells whether the given comment is fear 😨, Anger 😡, Joy 😂.

- As there are only 3 classes, this problem comes under the **Multi-Class Classification.**

In [1]:
#import pandas library
import pandas as pd

#read the dataset with name "Emotion_classify_Data.csv" and store it in a variable df
df = pd.read_csv("Emotion_classify_Data.csv")

#print the shape of dataframe

df.shape
#print top 5 rows
df.head()

,Comment,Emotion
0,i seriously hate one subject to death but now ...,fear
1,im so full of life i feel appalled,anger
2,i sit here to write i start to dig out my feel...,fear
3,ive been really angry with r and i feel like a...,joy
4,i feel suspicious if there is no one outside l...,fear


In [2]:
#check the distribution of Emotion
df.Emotion.value_counts()

Emotion
anger    2000
joy      2000
fear     1937
Name: count, dtype: int64

In [3]:
min_samples = 1937
df_anger = df[df.Emotion == "anger"].sample(min_samples, random_state=2022)
df_joy = df[df.Emotion == "joy"].sample(min_samples,random_state=2022)
df_fear = df[df.Emotion == "fear"].sample(min_samples,random_state=2022)

df = pd.concat([df_anger,df_joy,df_fear],axis=0)
df.Emotion.value_counts()

Emotion
anger    1937
joy      1937
fear     1937
Name: count, dtype: int64

In [4]:
#Add the new column "Emotion_num" which gives a unique number to each of these Emotions
#joy --> 0, fear --> 1, anger --> 2
df.Emotion = df.Emotion.map({
    "anger":0,
    "fear": 1,
    "joy": 2
})

#checking the results by printing top 5 rows
df.head()

,Comment,Emotion
2973,i constantly feel lied to and wronged by them ...,0
3785,i feel hated i feel like i dont belong and mor...,0
1860,i could ingrain in my mind all my feelings all...,0
5478,i dwell on this matter the more i feel infuria...,0
4939,i just feel too stubborn to give up on a dream,0


### **Modelling without Pre-processing Text data**

In [11]:
#import train-test split
from sklearn.model_selection import train_test_split

#Do the 'train-test' splitting with test size of 20%
#Note: Give Random state 2022 and also do the stratify sampling
X_train, X_test,y_train, y_test = train_test_split(
    df.Comment,
    df.Emotion,
    test_size=0.2,
    random_state=2022,
    stratify=df.Emotion
)


In [12]:
#print the shapes of X_train and X_test
print(
X_test.shape

)
print(X_train.shape)
print(y_train.shape
      )


(1163,)
(4648,)
(4648,)



**Attempt 1** :

1. using the sklearn pipeline module create a classification pipeline to classify the Data.

**Note:**
- using CountVectorizer with only trigrams.
- use **RandomForest** as the classifier.
- print the classification report.


              precision    recall  f1-score   support

           0       0.86      0.90      0.88       388
           1       0.85      0.89      0.87       388
           2       0.91      0.83      0.87       387

    accuracy                           0.87      1163
   macro avg       0.87      0.87      0.87      1163
weighted avg       0.87      0.87      0.87      1163




**Attempt 2** :

1. using the sklearn pipeline module create a classification pipeline to classify the Data.

**Note:**
- using CountVectorizer with both unigram and bigrams.
- use **Multinomial Naive Bayes** as the classifier.
- print the classification report.


In [ ]:
#import CountVectorizer, Multinomial Naive Bayes, pipeline, classification_report from sklearn 
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
#1. create a pipeline object
clf = Pipeline([
    ("vectorizer", TfidfVectorizer()),
    ("MNB" ,MultinomialNB()),
])


#2. fit with X_train and y_train
clf.fit(X_train,y_train)


#3. get the predictions for X_test and store it in y_pred
y_pred = clf.predict(X_test)


#4. print the classfication report
print(classification_report(y_test,y_pred))

<h3>Use text pre-processing to remove stop words, punctuations and apply lemmatization </h3>

In [14]:
import spacy

# load english language model and create nlp object from it
nlp = spacy.load("en_core_web_sm") 


#use this utility function to get the preprocessed text data
def preprocess(text):
    # remove stop words and lemmatize the text
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
    
    return " ".join(filtered_tokens) 

In [15]:
# create a new column "preprocessed_comment" and use the utility function above to get the clean data
# this will take some time, please be patient

df["preprocessedComment"] = df["Comment"].apply(preprocess)
df.head()

,Comment,Emotion,preprocessedComment
2973,i constantly feel lied to and wronged by them ...,0,constantly feel lied wrong love people death
3785,i feel hated i feel like i dont belong and mor...,0,feel hated feel like not belong feel want die
1860,i could ingrain in my mind all my feelings all...,0,ingrain mind feeling experience read hate happ...
5478,i dwell on this matter the more i feel infuria...,0,dwell matter feel infuriate m lowly thought
4939,i just feel too stubborn to give up on a dream,0,feel stubborn dream


**Build a model with pre processed text**

In [16]:
#Do the 'train-test' splitting with test size of 20% with random state of 2022 and stratify sampling too
#Note: Use the preprocessed_Comment
#import train-test split
from sklearn.model_selection import train_test_split

#Do the 'train-test' splitting with test size of 20%
#Note: Give Random state 2022 and also do the stratify sampling
X_train, X_test,y_train, y_test = train_test_split(
    df.preprocessedComment,
    df.Emotion,
    test_size=0.2,
    random_state=2022,
    stratify=df.Emotion
)


**Let's check the scores with our best model till now**
- Random Forest

**Attempt1** :

1. using the sklearn pipeline module create a classification pipeline to classify the Data.

**Note:**
- using CountVectorizer with both unigrams and bigrams.
- use **RandomForest** as the classifier.
- print the classification report.


In [17]:
#import CountVectorizer, Multinomial Naive Bayes, pipeline, classification_report from sklearn 
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
#1. create a pipeline object
clf = Pipeline([
    ("vectorizer", CountVectorizer()),
    ("MNB" ,RandomForestClassifier()),
])


#2. fit with X_train and y_train
clf.fit(X_train,y_train)


#3. get the predictions for X_test and store it in y_pred
y_pred = clf.predict(X_test)


#4. print the classfication report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.93      0.92       388
           1       0.93      0.91      0.92       388
           2       0.95      0.95      0.95       387

    accuracy                           0.93      1163
   macro avg       0.93      0.93      0.93      1163
weighted avg       0.93      0.93      0.93      1163




**Attempt 2** :

1. using the sklearn pipeline module create a classification pipeline to classify the data.

**Note:**
- using **TF-IDF vectorizer** for pre-processing the text.
- use **RandomForest** as the classifier.
- print the classification report.


In [18]:

#1. create a pipeline object
clf = Pipeline([
    ("vectorizer", TfidfVectorizer()),
    ("MNB" ,RandomForestClassifier()),
])


#2. fit with X_train and y_train
clf.fit(X_train,y_train)


#3. get the predictions for X_test and store it in y_pred
y_pred = clf.predict(X_test)


#4. print the classfication report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.93      0.93       388
           1       0.93      0.92      0.92       388
           2       0.94      0.94      0.94       387

    accuracy                           0.93      1163
   macro avg       0.93      0.93      0.93      1163
weighted avg       0.93      0.93      0.93      1163

